In [1]:
#import data handling libraries
import pandas as pd
import numpy as np

#import visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

#import machine learning libraries
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

from sklearn.feature_selection import SelectKBest, f_classif

In [2]:
df = pd.read_csv('lee_final_dataset_11_18.csv')
X = df.drop(['STUDENT','MAJOR','GRADUATE'], axis=1)
y = df['GRADUATE']

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25, 
                                                    random_state=42,
                                                    shuffle=True,
                                                    stratify=y)

In [3]:
classifiers = {
    # Putting linear decision boundary classifiers first
    'lda' : LinearDiscriminantAnalysis(),
    'log_reg' : LogisticRegression(penalty=None, max_iter= 100000),
     'svc_linear' : LinearSVC(dual = 'auto'),

    # Quadratic boundaries
    'qda' : QuadraticDiscriminantAnalysis(),
    'lda_poly' : Pipeline([('scale', StandardScaler()),('poly',PolynomialFeatures(2)),('lda', LinearDiscriminantAnalysis())]),
    'log_reg_poly' : Pipeline([('scale', StandardScaler()),('poly',PolynomialFeatures(2)),('log_reg', LogisticRegression(penalty=None, max_iter= 100000))]),
    'gnb' : GaussianNB(),

    # Complex boundaries
    'knn' : Pipeline([('scale', StandardScaler()),('knn', KNeighborsClassifier())]),   
    'svc_rbf' : Pipeline([('scale', StandardScaler()),('svc',SVC(kernel= 'rbf'))])
}
importances = {}

for model_name, model in classifiers.items():
    model.fit(X_train, y_train)
    if hasattr(model, 'named_steps'):
        # For pipelines, get the final step
        final_model = model.named_steps[list(model.named_steps.keys())[-1]]
    else:
        final_model = model
    
    if hasattr(final_model, 'feature_importances_'):
        importances[model_name] = final_model.feature_importances_
    elif hasattr(final_model, 'coef_'):
        importances[model_name] = final_model.coef_[0]  # For linear models
        
# Print feature importances or coefficients ordered by absolute value
for model_name, importance in importances.items():
    print(f"\nFeature importances for {model_name}:")
    sorted_importances = sorted(zip(X.columns, importance), key=lambda x: abs(x[1]), reverse=True)
    for feature_name, value in sorted_importances:
        print(f"{feature_name}: {value}")


Feature importances for lda:
160: 0.22797166331827395
104: 0.17451267608440163
304: 0.14874930032038952
140: 0.14867632263808128
266: 0.11679489880793657
150: 0.1023238875604928
414: 0.09755342156682675
435: 0.08725676953779374
207: 0.08496605362648374
373: 0.0791451091282421
105: 0.07726782506993511
385: 0.07233873179630504
302: 0.0671856045443982
317: 0.061355721350489056
143: -0.05172650594308262
350: 0.04273363166305006
201: -0.04220473651934754
314: 0.03970949813640744
436: -0.031453334977110614
151: 0.025763038661751746
415: 0.02226939174097745
265: 0.02137312441309826
145: -0.02086539073087955
166: -0.010185280843562253
301: 0.009967389269456183
365: -0.006888295192371208
267: -0.005518766732736198
165: -0.0033297969044547484

Feature importances for log_reg:
160: 0.24468549802374842
304: 0.23611000074694732
104: 0.18305614445091364
140: 0.15896958947704576
435: 0.12584034790053963
266: 0.12270738045269654
414: 0.11610079292829915
373: 0.112833378860537
302: 0.10547201013102492

In [4]:
cv_results = {}
for model_name, model in classifiers.items():
    scores = cross_val_score(model, 
                             X_train, 
                             y_train, 
                             cv=5,
                             scoring='accuracy')
    cv_results[model_name] = scores
    print(f"{model_name}: Mean -logloss = {scores.mean():.4f}, Std = {scores.std():.4f}")

# Print the cross-validation results
for model_name, scores in cv_results.items():
    print(f"{model_name}: Mean -logloss = {scores.mean():.4f}, Std = {scores.std():.4f}")

lda: Mean -logloss = 0.6158, Std = 0.0105
log_reg: Mean -logloss = 0.6155, Std = 0.0096
svc_linear: Mean -logloss = 0.6159, Std = 0.0107
qda: Mean -logloss = 0.5465, Std = 0.0138
lda_poly: Mean -logloss = 0.6317, Std = 0.0193
log_reg_poly: Mean -logloss = 0.6259, Std = 0.0180
gnb: Mean -logloss = 0.5394, Std = 0.0137
knn: Mean -logloss = 0.6052, Std = 0.0143
svc_rbf: Mean -logloss = 0.6263, Std = 0.0123
lda: Mean -logloss = 0.6158, Std = 0.0105
log_reg: Mean -logloss = 0.6155, Std = 0.0096
svc_linear: Mean -logloss = 0.6159, Std = 0.0107
qda: Mean -logloss = 0.5465, Std = 0.0138
lda_poly: Mean -logloss = 0.6317, Std = 0.0193
log_reg_poly: Mean -logloss = 0.6259, Std = 0.0180
gnb: Mean -logloss = 0.5394, Std = 0.0137
knn: Mean -logloss = 0.6052, Std = 0.0143
svc_rbf: Mean -logloss = 0.6263, Std = 0.0123
